<a href="https://colab.research.google.com/github/shimazarghami/DeepNN/blob/main/mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2

cap = cv2.VideoCapture("test2.mp4")

print("Is opened:", cap.isOpened())
cap.release()

In [ ]:
import cv2, os

os.makedirs("frames2", exist_ok=True)

cap = cv2.VideoCapture("test2.mp4")
frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_id % 5 == 0:  # هر 5 فریم یکی
        cv2.imwrite(f"frames2/frame_{frame_id}.jpg", frame)

    frame_id += 1

cap.release()


In [ ]:
pip install git+https://github.com/facebookresearch/segment-anything-2.git


In [ ]:
pip install ultralytics


In [ ]:
from ultralytics import YOLO
import cv2
import os

# 1. Load pretrained YOLO
model = YOLO("yolo11n.pt")   # خیلی سبک و سریع

frames_dir = "frames2"
labels_dir = "auto_labels2"

os.makedirs(labels_dir, exist_ok=True)

for img_name in os.listdir(frames_dir):
    if not img_name.endswith(".jpg"):
        continue

    img_path = os.path.join(frames_dir, img_name)

    results = model(img_path, conf=0.4)

    img = cv2.imread(img_path)
    h, w = img.shape[:2]

    label_path = os.path.join(
        labels_dir, img_name.replace(".jpg", ".txt")
    )

    with open(label_path, "w") as f:
        for box in results[0].boxes.xyxy:
            x1, y1, x2, y2 = box.tolist()

            # YOLO format
            x_c = ((x1 + x2) / 2) / w
            y_c = ((y1 + y2) / 2) / h
            bw = (x2 - x1) / w
            bh = (y2 - y1) / h

            f.write(f"0 {x_c} {y_c} {bw} {bh}\n")

print("✅ Auto-labeling finished.")


In [ ]:
import os, random, shutil

images_src = "frames2"
labels_src = "auto_labels2"

img_train = "dataset/images/train"
img_val   = "dataset/images/val"
lbl_train = "dataset/labels/train"
lbl_val   = "dataset/labels/val"

for p in [img_train, img_val, lbl_train, lbl_val]:
    os.makedirs(p, exist_ok=True)

files = [f for f in os.listdir(images_src) if f.endswith(".jpg")]
random.shuffle(files)

split = int(0.8 * len(files))

for i, f in enumerate(files):
    src_img = os.path.join(images_src, f)
    src_lbl = os.path.join(labels_src, f.replace(".jpg", ".txt"))

    if i < split:
        shutil.copy(src_img, img_train)
        shutil.copy(src_lbl, lbl_train)
    else:
        shutil.copy(src_img, img_val)
        shutil.copy(src_lbl, lbl_val)

print("✅ Dataset split completed")


In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")

model.train(
    data="dataset/data.yaml",
    epochs=30,
    imgsz=640,
    batch=16
)


In [ ]:
from ultralytics import YOLO
import supervision as sv
import cv2

model = YOLO("runs/detect/train/weights/best.pt")

cap = cv2.VideoCapture("video/conveyor.mp4")

line = sv.LineZone(
    start=sv.Point(100, 300),
    end=sv.Point(600, 300)
)

annotator = sv.LineZoneAnnotator(thickness=2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)

    line.trigger(detections)
    frame = annotator.annotate(frame, line)

    cv2.imshow("Object Counting", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
model("frames2/frame_10.jpg", conf=0.25, save=True)